In [ ]:
# Credit Risk Probability Model - Exploratory Data Analysis

This notebook contains exploratory analysis of the eCommerce transaction data to develop insights for the credit risk model. The goal is to identify patterns, data quality issues, and form hypotheses to guide feature engineering.

## Task Overview
We need to develop a credit scoring model that utilizes eCommerce behavioral data to assess credit risk for a buy-now-pay-later service. This requires:

1. Defining a proxy variable for credit risk categorization (high/low risk)
2. Selecting predictive features that correlate with default risk
3. Building models to assign risk probability and credit scores


In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import datetime as dt

# Set visualization options
plt.style.use('ggplot')
sns.set_palette('viridis')
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
%matplotlib inline


In [ ]:
## 1. Loading and Exploring the Data Structure

The data is available from the Xente Challenge on Kaggle. Here we will load the dataset and explore its structure to understand the available features and data types.

**Note:** In a real-world scenario, you would need to download the data from the provided link and place it in the `data/raw` directory. For this notebook, we'll use a placeholder path that you should update with the actual location of your data file.


In [ ]:
# Load the data
# Update this path to the location of your data file
data_file = "../data/raw/data.csv"

try:
    # Try to load the data if the file exists
    df = pd.read_csv(data_file)
    print(f"Data loaded successfully with {df.shape[0]} rows and {df.shape[1]} columns")
except FileNotFoundError:
    print(f"Data file not found at {data_file}")
    print("Please download the data from the Xente Challenge on Kaggle and place it in the data/raw directory")
    # Create a sample dataframe with the expected columns for demonstration
    df = pd.DataFrame({
        'TransactionId': ['T1001', 'T1002', 'T1003'],
        'BatchId': [1, 1, 2],
        'AccountId': ['A001', 'A002', 'A001'],
        'SubscriptionId': ['S001', 'S002', 'S001'],
        'CustomerId': ['C001', 'C002', 'C001'],
        'CurrencyCode': ['UGX', 'UGX', 'UGX'],
        'CountryCode': [123, 123, 123],
        'ProviderId': ['P001', 'P002', 'P003'],
        'ProductId': ['PROD001', 'PROD002', 'PROD003'],
        'ProductCategory': ['Airtime', 'Utility', 'Airtime'],
        'ChannelId': ['web', 'android', 'web'],
        'Amount': [5000, -2000, 10000],
        'Value': [5000, 2000, 10000],
        'TransactionStartTime': ['2018-10-01 10:10:10', '2018-10-01 11:10:10', '2018-10-02 10:10:10'],
        'PricingStrategy': ['A', 'B', 'A'],
        'FraudResult': [0, 1, 0]
    })


In [ ]:
# Examine the structure of the data
print("Data types for each column:")
print(df.dtypes)
print("\nFirst 5 rows of the dataset:")
df.head()


In [ ]:
# Basic information about the dataset
print("Dataset info:")
df.info()


In [ ]:
## 2. Summary Statistics

Let's calculate summary statistics for the numerical features to understand their central tendencies, dispersion, and shape.


In [ ]:
# Summary statistics for numerical variables
print("Summary statistics for numerical variables:")
df.describe().T


In [ ]:
# Convert transaction time to datetime
if 'TransactionStartTime' in df.columns:
    df['TransactionStartTime'] = pd.to_datetime(df['TransactionStartTime'])
    
    # Extract time-based features
    df['TransactionYear'] = df['TransactionStartTime'].dt.year
    df['TransactionMonth'] = df['TransactionStartTime'].dt.month
    df['TransactionDay'] = df['TransactionStartTime'].dt.day
    df['TransactionDayOfWeek'] = df['TransactionStartTime'].dt.dayofweek
    df['TransactionHour'] = df['TransactionStartTime'].dt.hour
    
    print("Added datetime features from TransactionStartTime")


In [ ]:
## 3. Distribution of Numerical Features

Let's visualize the distributions of key numerical features to identify patterns, skewness, and potential outliers.


In [ ]:
# Plot distributions of Amount and Value
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Amount distribution
if 'Amount' in df.columns:
    sns.histplot(df['Amount'], kde=True, ax=axes[0])
    axes[0].set_title('Distribution of Amount')
    axes[0].set_xlabel('Amount')
    
    # Value distribution
    if 'Value' in df.columns:
        sns.histplot(df['Value'], kde=True, ax=axes[1])
        axes[1].set_title('Distribution of Value')
        axes[1].set_xlabel('Value')
        
plt.tight_layout()
plt.show()


In [ ]:
# Box plots for numerical features to identify outliers
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
if len(numerical_cols) > 0:
    # Select a subset of numerical columns for boxplot
    plot_cols = [col for col in numerical_cols if col not in ['CustomerId', 'BatchId']][:4]  # Limit to 4 columns
    
    if plot_cols:
        plt.figure(figsize=(14, 6))
        sns.boxplot(data=df[plot_cols])
        plt.title('Boxplots of Numerical Features')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
else:
    print("No numerical columns found for boxplot visualization")


In [ ]:
## 4. Distribution of Categorical Features

Let's analyze the distribution of categorical features to understand their frequency and variability.


In [ ]:
# Identify categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

# Remove datetime columns from categorical columns if any
if 'TransactionStartTime' in categorical_cols:
    categorical_cols.remove('TransactionStartTime')

# Print unique values for each categorical column
for col in categorical_cols:
    print(f"\nColumn: {col}")
    print(f"Number of unique values: {df[col].nunique()}")
    print("Value counts:")
    display(df[col].value_counts().head(10))  # Display top 10 values


In [ ]:
# Visualize distribution of key categorical variables
key_categorical = ['ProductCategory', 'ChannelId', 'PricingStrategy'] 
key_categorical = [col for col in key_categorical if col in df.columns]

if key_categorical:
    fig, axes = plt.subplots(len(key_categorical), 1, figsize=(12, 5*len(key_categorical)))
    
    # Handle case of one categorical variable
    if len(key_categorical) == 1:
        axes = [axes]
        
    for i, col in enumerate(key_categorical):
        value_counts = df[col].value_counts()
        # If there are too many categories, limit to top 10
        if len(value_counts) > 10:
            value_counts = value_counts.nlargest(10)
            
        sns.barplot(x=value_counts.index, y=value_counts.values, ax=axes[i])
        axes[i].set_title(f'Distribution of {col}')
        axes[i].set_ylabel('Count')
        axes[i].set_xlabel(col)
        axes[i].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()


In [ ]:
## 5. Missing Values Analysis

Let's check for missing values in the dataset and decide on appropriate imputation strategies if needed.


In [ ]:
# Calculate missing values
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

# Create a dataframe with the results
missing_info = pd.DataFrame({
    'Missing Values': missing_values,
    'Missing Percentage': missing_percentage
})

# Display only columns with missing values
missing_info = missing_info[missing_info['Missing Values'] > 0].sort_values('Missing Percentage', ascending=False)

if len(missing_info) > 0:
    print("Columns with missing values:")
    display(missing_info)
else:
    print("No missing values found in the dataset!")


In [ ]:
## 6. Correlation Analysis

Let's examine the correlation between numerical features to identify potential relationships and dependencies.


In [ ]:
# Select numerical features for correlation analysis
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns

# Ensure there are enough numerical variables for meaningful correlation
if len(numerical_cols) >= 2:
    # Calculate correlation matrix
    correlation_matrix = df[numerical_cols].corr()
    
    # Plot correlation heatmap
    plt.figure(figsize=(12, 10))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
    plt.title('Correlation Heatmap of Numerical Features')
    plt.tight_layout()
    plt.show()
    
    # Identify highly correlated pairs (|corr| > 0.7)
    high_corr_pairs = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i):
            if abs(correlation_matrix.iloc[i, j]) > 0.7:
                high_corr_pairs.append((correlation_matrix.columns[i], correlation_matrix.columns[j], 
                                      correlation_matrix.iloc[i, j]))
                
    if high_corr_pairs:
        print("Highly correlated feature pairs (|corr| > 0.7):")
        for col1, col2, corr in high_corr_pairs:
            print(f"{col1} - {col2}: {corr:.3f}")
    else:
        print("No highly correlated feature pairs found.")
else:
    print("Not enough numerical columns for correlation analysis")


In [ ]:
## 7. FraudResult Analysis

Since FraudResult could be a potential proxy for credit risk, let's analyze its distribution and relationship with other variables.


In [ ]:
# Check if FraudResult is available in the dataset
if 'FraudResult' in df.columns:
    # Distribution of FraudResult
    print("Distribution of FraudResult:")
    fraud_counts = df['FraudResult'].value_counts()
    print(fraud_counts)
    
    # Calculate percentage of each class
    fraud_percentage = (fraud_counts / len(df)) * 100
    print("\nPercentage distribution:")
    for label, percentage in fraud_percentage.items():
        print(f"FraudResult = {label}: {percentage:.2f}%")
    
    # Visualize FraudResult distribution
    plt.figure(figsize=(10, 6))
    ax = sns.countplot(x='FraudResult', data=df)
    plt.title('Distribution of FraudResult')
    
    # Add count labels on top of each bar
    for p in ax.patches:
        ax.annotate(f'{int(p.get_height())}', 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'bottom')
    
    plt.show()
else:
    print("FraudResult column not found in the dataset")


In [ ]:
# Analyze FraudResult by key features
if 'FraudResult' in df.columns:
    # Key features to analyze against FraudResult
    key_features = ['ProductCategory', 'ChannelId', 'PricingStrategy']
    key_features = [col for col in key_features if col in df.columns]
    
    if key_features:
        fig, axes = plt.subplots(len(key_features), 1, figsize=(12, 6*len(key_features)))
        
        # Handle case of one feature
        if len(key_features) == 1:
            axes = [axes]
            
        for i, feature in enumerate(key_features):
            fraud_by_feature = pd.crosstab(df[feature], df['FraudResult'], normalize='index') * 100
            fraud_by_feature.plot(kind='bar', ax=axes[i], stacked=False)
            axes[i].set_title(f'Fraud Rate by {feature}')
            axes[i].set_ylabel('Percentage (%)')
            axes[i].set_xlabel(feature)
            axes[i].legend(title='FraudResult')
            
            # Add percentage labels
            for p in axes[i].patches:
                width = p.get_width()
                height = p.get_height()
                x, y = p.get_xy() 
                if height > 5:  # Only add label if segment is large enough
                    axes[i].annotate(f'{height:.1f}%', (x + width/2, y + height/2), 
                                   ha='center', va='center')
        
        plt.tight_layout()
        plt.show()
else:
    print("FraudResult column not found in the dataset")


In [ ]:
## 8. RFM Analysis

Creating RFM (Recency, Frequency, Monetary) metrics will be crucial for our credit risk model. Let's compute these metrics and analyze their distributions.


In [ ]:
# Create RFM metrics if necessary data is available
if 'TransactionStartTime' in df.columns and 'CustomerId' in df.columns and ('Amount' in df.columns or 'Value' in df.columns):
    # Use the most recent date as a reference point
    if len(df) > 0:
        snapshot_date = df['TransactionStartTime'].max() + pd.Timedelta(days=1)
        
        # Monetary value - use Value instead of Amount as Amount can be negative
        monetary_col = 'Value' if 'Value' in df.columns else 'Amount'
        
        # Calculate RFM metrics per customer
        rfm = df.groupby('CustomerId').agg({
            'TransactionStartTime': lambda x: (snapshot_date - x.max()).days,  # Recency
            'TransactionId': 'count',  # Frequency
            monetary_col: 'sum'  # Monetary
        }).rename(columns={
            'TransactionStartTime': 'Recency',
            'TransactionId': 'Frequency',
            monetary_col: 'Monetary'
        })
        
        # Display RFM metrics
        print("RFM Metrics for Customers:")
        display(rfm.head())
        
        # Summary statistics for RFM metrics
        print("\nRFM Summary Statistics:")
        display(rfm.describe())
        
        # Visualize distributions
        plt.figure(figsize=(18, 6))
        
        # Recency distribution
        plt.subplot(1, 3, 1)
        sns.histplot(rfm['Recency'], kde=True)
        plt.title('Recency Distribution')
        plt.xlabel('Recency (days)')
        
        # Frequency distribution 
        plt.subplot(1, 3, 2)
        sns.histplot(rfm['Frequency'], kde=True)
        plt.title('Frequency Distribution')
        plt.xlabel('Frequency (count)')
        
        # Monetary distribution
        plt.subplot(1, 3, 3)
        sns.histplot(rfm['Monetary'], kde=True)
        plt.title('Monetary Distribution')
        plt.xlabel('Monetary Value')
        
        plt.tight_layout()
        plt.show()
    else:
        print("Empty DataFrame, cannot calculate RFM metrics")
else:
    print("Missing required columns for RFM analysis (TransactionStartTime, CustomerId, Amount/Value)")


In [ ]:
## 9. Key Insights and Next Steps

Based on our exploratory data analysis, here are the key insights and proposed next steps for feature engineering and modeling.


In [ ]:
# Summary of Key Insights

"""
Key Insights from EDA:

1. **Proxy Variable Definition**: 
   - The FraudResult column provides a potential starting point for creating our credit risk proxy
   - We may need to combine this with RFM metrics to create a more comprehensive risk proxy
   - The class imbalance in FraudResult suggests we'll need appropriate handling techniques

2. **Feature Importance**:
   - Transactional features like Amount and Value show significant variation and potential predictive power
   - Categorical variables (ProductCategory, ChannelId, PricingStrategy) show varying distributions of fraud
   - Time-based patterns may provide strong signals for credit risk prediction  

3. **RFM Analysis**:
   - Customer behavior patterns captured through RFM metrics show distinct segments
   - The monetary distribution indicates potential outliers that may need special handling
   - Frequency patterns could be strong predictors of creditworthiness

4. **Data Quality**:
   - [Note any missing values or data quality issues identified]
   - [Note any unusual distributions or outliers discovered]
   - [Note any inconsistencies or anomalies in the data]

5. **Relationships and Correlations**:
   - Strong correlation between [highlight any strong correlations discovered]
   - Potential multicollinearity between [mention any concerning correlations]
   - Nonlinear relationships between [mention any interesting nonlinear patterns]

Next Steps for Feature Engineering:
1. Create comprehensive RFM features per customer
2. Engineer temporal features (day of week patterns, seasonality, etc.)
3. Develop behavioral segments based on transaction patterns
4. Transform skewed numerical features
5. Create a robust proxy variable for credit risk by combining FraudResult with RFM metrics
"""

print("""Note: This is a placeholder analysis based on the expected structure of the dataset. 
When working with the actual data, you should replace these insights with specific findings from your analysis.""")
